In [4]:
# Complete tinyGPT
# Then build and train gpt2
# Papers - Attention is all you need
# GPT 2/3 paper

# Llamac/llama2.c
# Cuda 

In [5]:
# how to?
# Model arcthitecture -> 
# Data(find and filter) -> 
# train -> 
# testing/benchmarks -> 
# Sample
# Some other evaluations and researches like scaling laws, transformer sizing
# Configurator
# Play with some opensource model also
# pretraining | post training

In [6]:
from transformers import GPT2LMHeadModel

In [7]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
sd_hf = model_hf.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)


transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [8]:
# we train model from as small as one to context length
# so model will see all kinds of context length and from 1 to length of context(block size)
# when predicting the character
# batch - for parallel processing
# In multidimensional matrix
# Each dimension is axis and each axis has index
#dim=0 → collapsing rows → gives a summary per column
# dim=1 → collapsing columns → gives a summary per row
# [a, b, c] = [a][b][c] : same in slicing

In [9]:
# Lets create a BigramNeuralNetwork class with attention in it
vocab_size = 8
n_embd = 2
import torch
import torch.nn as nn
class BigramNeuralNetwork(torch.nn):
    def __init__(self):
        # define how the neural network looks like
        # their structure
        # input layer
        # middle layer
        # output layer
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx):
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        logits = self.lm_head(tok_emb) # (B, T, vocab_size)
    

TypeError: module() takes at most 2 arguments (3 given)

In [ ]:
# Reading Attention is all you need paper
class Block:

    pass

In [ ]:
# B - Batch - parallel input sequence
# T - Time(vocab size, sequence length, memory used in prediction, Context size) - position of tokens
# c - channel(embedding dimensions, d_model) - Contains feature vector for each token
# torch.tril(torch.ones(T, T))
# embedding table contains feature vector for each token
# each row corresponds to a token and columns feature vector
# embedding table dimensions - (vocab_size, Channel)
# Each token becomes three vectors: 
# Query: what this token is looking for?
# Key: what this token has to offer
# Value: what the information this token contains for sharing to next token i thinks
# Encoder model: Takes all the tokens in consideration
# Decoder model: Takes only current and past tokens in consideration
# Query @ Key = Scores
# Scale - Stabilize scores
# Softmax(scores) = weights 
# Weighted average of values - New token representations
# Scores: strong positive value - strong match high attention
# Near zero: No match - Low attention
# Negative number: Disimilar - very low ignored after softmax
# Linear Layer is simple matrix multiplication- input @ weight + bias = w @ x + b
# Training decides that what query, key and value will contain
# Mainly backpropagation and gradient descent this is amazing
# embedding dimensions or channel dimensions or feature vector of a token
# is broken down in to key, query and vector and we train these three things
# and after training these things contains meaning representation of embedding dimensions
# query of current token, and key and values of all the past token
# Keep the dimensions of k, q, v so dot products should be easy
# Mostly same or half
# in multi head attention keep the dimension as embd_d // head_size
#  The dot product
# the dot prodcut in between |A||B|cos(theta)
# In Other words: Projection of a on b
# IOW: how much A aligns with the B
# iow: how much a is in the direction of B
# iow: similarity between two vectors

import torch
B, T, C = 4, 8, 32 # consider this as stacked sheet of paper
x = torch.randn(B, T, C)
key = nn.Linear(32, 16)
print(key.weight.shape)
query = nn.Linear(32, 16)
k = key(x) # 4, 8, 16
q = query(x) # 4, 8, 16
out = (k @ q.transpose(-2, -1)) / k[-1].shape ** 0.5 # (4, 8, 16) @ (4, 16, 8) -> 4, 8, 8
print(out.shape)
# print(key(x).shape) # 4, 8, 32 @ 4, 32, 16 ---> 4, 8, 16 
# ex = nn.Linear(3, 5)
# print(x2.weight)
# x11 = x1(x)
# y11 = x2(x)
# y11.shape
# x11 @ y11

torch.Size([16, 32])
torch.Size([4, 8, 8])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
B, T, C = 4, 8, 32
vocab_size = 64 # How many tokens?
batch_size = 4 # input sequence we will processed in parallel
block_size = 8 # Context length
embd_d = 32 # Feature vector to represent a token
head_size = 16 # size of head
x = torch.randn(B, T, C) # Input data

# One Head Attention Block
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embd_d, head_size, bias=False) # y = x @ wT + b
        self.query = nn.Linear(embd_d, head_size, bias=False)
        self.value = nn.Linear(embd_d, head_size, bias=False)
        # to make lower triangle that will help in weighted average calculation
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size))) 
    
    def forward(self, x):
        q = self.query(x) # (B, T, head_size)
        k = self.key(x) # (B, T, head_size)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** 0.5 # (B, T, head_size) @ (B, head_size) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf")) # (B, T, T), deactivating next tokens
        wei = F.softmax(wei, dim= -1) # (B, T, T)
        # Weighted aggregation of the values
        v = self.value(x) # (B, T, head_size)
        out = wei @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
        return out
attention = Head(head_size)
y = attention(x)
print(y.shape)
print(y[0])

torch.Size([4, 8, 16])
tensor([[-0.5116,  0.1790,  0.1477,  0.4020, -0.5156,  0.0225, -0.2813,  0.0107,
          0.6123, -0.4112, -0.2374,  0.3414, -0.2318,  0.0701,  0.1628, -1.2048],
        [-0.4916,  0.1789,  0.1393,  0.3719, -0.5013,  0.0254, -0.2581,  0.0073,
          0.5818, -0.3724, -0.2384,  0.2642, -0.2258,  0.0303,  0.1463, -1.0543],
        [-0.2618,  0.1780,  0.0428,  0.0266, -0.3375,  0.0586,  0.0081, -0.0322,
          0.2317,  0.0728, -0.2498, -0.6218, -0.1567, -0.4260, -0.0434,  0.6724],
        [-0.4935,  0.1789,  0.1402,  0.3750, -0.5027,  0.0251, -0.2604,  0.0076,
          0.5849, -0.3763, -0.2383,  0.2720, -0.2264,  0.0343,  0.1480, -1.0696],
        [-0.2504,  0.1766,  0.2284,  0.3013, -0.4933, -0.0036, -0.1632, -0.1655,
          0.5580, -0.2457, -0.2732,  0.3370, -0.1941,  0.0295,  0.1574, -1.0176],
        [ 1.1250,  0.2218,  1.1225, -0.0294, -0.3124,  0.3271, -0.0952, -1.3758,
          0.1166,  0.2256, -0.4738,  0.3247,  0.1788, -0.0392,  0.0256, -0.4580],